----
<div style="display: flex; align-items: center;">
    <img src="https://developers.lseg.com/content/dam/devportal/icons/logo/lseg-logo.svg" width="20%" style="vertical-align: top;">
    <h1 style="margin-left: 20px;">Data Library for Python</h1>
</div>

----

## Delivery - OMM Stream - Market By Price data via callback

This notebook demonstrates how to use the OMM Stream interface to request streaming Full Depth Orderbook data

## Set the location of the configuration file
For ease of use, you can set various initialization parameters of the RD Library in the **_lseg-data.config.json_** configuration file - as described in the Quick Start -> Sessions example.

### One config file for the tutorials
As these tutorial Notebooks are categorised into sub-folders and to avoid the need for multiple config files, we will use the _LD_LIB_CONFIG_PATH_ environment variable to point to a single instance of the config file in the top-level ***Configuration*** folder.

Before proceeding, please **ensure you have entered your credentials** into the config file in the ***Configuration*** folder.

In [1]:
import os
os.environ["LD_LIB_CONFIG_PATH"] = "../../../Configuration"

In [1]:
from lseg.data.delivery import omm_stream
import lseg.data as rd
import datetime
import json

## Open the default session

To open the default session ensure you have a '*lseg-data.config.json*' in the ***Configuration*** directory, populated with your credentials and specified a 'default' session in the config file    


In [2]:
rd.open_session()

<lseg.data.session.Definition object at 0x2294fed0150 {name='workspace'}>

# Example

## Define a function to display events

In [3]:
# Callback function to display data or status events
def display_event(eventType, event):
    currentTime = datetime.datetime.now().time()
    print("----------------------------------------------------------")
    print(">>> {} event received at {}".format(eventType, currentTime))
    # Print only 1000 characters to limit the output
    print(json.dumps(event, indent=2)[0:1000])
    return

## Create an OMM Stream and register the event callbacks

In [4]:
stream = omm_stream.Definition(name="VOD.L", domain='MarketByPrice').get_stream()

# Refresh - the first full imaage we get back from the server
stream.on_refresh(lambda event, stream : display_event("Refresh", event))
# Update - as and when field values change, we receive updates from the server
stream.on_update(lambda event, stream : display_event("Update", event))
# Status - if data goes stale or item closes, we get a status message
stream.on_status(lambda event, stream : display_event("Status", event))
# Other errors
stream.on_error(lambda event, stream : display_event("Error", event))

## Open the Item Stream
The **open()** call to open the OMM Stream is a synchronous one. This means the first event (either via on_refresh(), on_status() or on_error()) can occur before the open() method returns.   
However, if we use the **open_async()** asynchronous method instead, the first event callback will be after  open_async() returns.

In [5]:
# Library will request OrderBook from server
stream.open()
# We should intially receive the full orderbook, 
# after which we will receive updates for specific order (Add, Update, Delete)
# Note from the above display_event() function that I am dumping just the first 1000 characters to minimise output
# You should remove this limit to see the full response.

----------------------------------------------------------

<OpenState.Opened: 'Opened'>


>>> Status event received at 18:45:58.335993
{
  "Domain": "MarketByPrice",
  "Type": "Status",
  "ID": 5,
  "Key": {
    "Name": "VOD.L"
  },
  "State": {
    "Data": "Suspect",
    "Stream": "Closed",
    "Code": "NotEntitled",
    "Text": "Access Denied: User req to PE(249)"
  }
}


You may notice that there are multiple Refresh events - this is often the case for the more actively traded instruments with large orders books.
Once all Refresh events have been received, you can then expect to receive Update events with just the Order changes i.e. Add, Update or Delete orders.  
**NOTE:** I am truncating the output to the 1st 1000 character of each response payload - for ease of viewing.  


## Close Stream

In [6]:
stream.close()

<OpenState.Closed: 'Closed'>

### Close Session

In [7]:
rd.close_session()